# 📃 DDL: Tabela Silver - `tb_chamado`

**Origem:** Tabela Bronze `chamados`
**Domínio:** Atendimento ao Cliente

## ⚙️ Decisões de Governança e Modelagem

### 1. Padronização de Nomenclatura
Aplicamos prefixos semânticos para facilitar a descoberta e uso dos dados:
- **`cd_` (Código):** Substitui `id_` para identificadores únicos e chaves estrangeiras (ex: `id_chamado` → `cd_chamado`).
- **`st_` (Status):** Para colunas booleanas ou de estado (ex: `resolvido` → `st_resolvido`).
- **`tm_` (Tempo/Medida):** Para métricas de duração (ex: `tempo_espera` → `tm_espera`).

### 2. Mapeamento de Schema (Bronze → Silver)
| Coluna Bronze | Coluna Silver | Tipo Novo | Justificativa |
|---|---|---|---|
| `id_chamado` | `cd_chamado` | `BIGINT` | PK da tabela. |
| `id_cliente` | `cd_cliente` | `STRING` | FK para `tb_cliente`. Mantido String pois é um Hash. |
| `id_motivo` | `cd_motivo` | `BIGINT` | FK para `tb_motivo`. |
| `id_canal` | `cd_canal` | `BIGINT` | FK para `tb_canal`. |
| `resolvido` | `st_resolvido` | `BOOLEAN` | Indicador de resolução direto (True/False). |
| `tempo_espera` | `tm_espera` | `BIGINT` | Métrica de SLA. |
| `tempo_duracao`| `tm_duracao` | `BIGINT` | Métrica de esforço. |

### 3. Constraints
- **Primary Key:** `pk_chamados` (`cd_chamado`).
- **Foreign Keys (Lógicas):** Relacionamentos garantidos via pipeline com `tb_cliente`, `tb_motivo`, `tb_canal` e `tb_atendente`.

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_chamado"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

In [0]:
query = f"""
CREATE OR REPLACE TABLE {nome_tabela_destino} (
    cd_chamado BIGINT COMMENT 'Identificador do chamado.',
    cd_cliente STRING COMMENT 'Referência ao cliente (Hash).',
    cd_motivo BIGINT COMMENT 'Indentificador do Motivo relatado.',
    cd_canal BIGINT COMMENT 'Identificador do canal.',
    st_resolvido BOOLEAN COMMENT 'Indicador de resolução (T/F).',
    tm_espera BIGINT COMMENT 'Tempo em fila até atendimento.',
    tm_duracao BIGINT COMMENT 'Duração total do atendimento.',
    cd_atendente BIGINT COMMENT 'Referência ao atendente.',
    dt_ingestion TIMESTAMP,
    dc_origem STRING,
    CONSTRAINT pk_chamados PRIMARY KEY (cd_chamado)
)
COMMENT 'Tabela silver de histórico de chamados';
"""

spark.sql(query)
print(f"✅ Tabela {nome_tabela_destino} criada com sucesso!")